In [32]:
import sys
import os
import pandas as pd # type: ignore
import numpy as np # type: ignore
from sklearn.preprocessing import StandardScaler # type: ignore
from sklearn.cluster import KMeans # type: ignore
from scipy.stats import zscore # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
sys.path.append(os.path.abspath('C:/Users/nejat/AIM Projects/Telecommunication Data Analysis/src'))
from db_connection import PostgresConnection
from data_cleaning import preprocess_data  

def load_data():
    query = "SELECT * FROM xdr_data"
    db = PostgresConnection(dbname='tellco', user='postgres', password='237132')
    db.connect()
    result = db.execute_query(query)
    df = pd.DataFrame(result, columns=[desc[0] for desc in db.cursor.description])
    db.close_connection()
    print(df.head()) 
    return df

df = load_data()

Connected to PostgreSQL database!
Connection closed.
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L7

In [7]:
print(df.columns)


Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [33]:
def clean_and_aggregate_data(df):
    df['TCP DL Retrans. Vol (Bytes)'].fillna(df['TCP DL Retrans. Vol (Bytes)'].mean(), inplace=True)
    df['Avg RTT DL (ms)'].fillna(df['Avg RTT DL (ms)'].mean(), inplace=True)
    df['Avg Bearer TP DL (kbps)'].fillna(df['Avg Bearer TP DL (kbps)'].mean(), inplace=True)
    df['Handset Type'].fillna(df['Handset Type'].mode()[0], inplace=True)
    
    for col in ['TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)']:
        lower_bound = df[col].quantile(0.01)
        upper_bound = df[col].quantile(0.99)
        df[col] = np.clip(df[col], lower_bound, upper_bound)
    
    agg_df = df.groupby('MSISDN/Number').agg({
        'TCP DL Retrans. Vol (Bytes)': 'mean',
        'Avg RTT DL (ms)': 'mean',
        'Avg Bearer TP DL (kbps)': 'mean',
        'Handset Type': 'first'
    }).reset_index()
    
    return agg_df


agg_df = clean_and_aggregate_data(df)
print(agg_df)

        MSISDN/Number  TCP DL Retrans. Vol (Bytes)  Avg RTT DL (ms)  \
0        3.360100e+10                 2.080991e+07        46.000000   
1        3.360100e+10                 2.080991e+07        30.000000   
2        3.360100e+10                 2.080991e+07       109.795706   
3        3.360101e+10                 1.066000e+03        69.000000   
4        3.360101e+10                 1.507977e+07        57.000000   
...               ...                          ...              ...   
106851   3.379000e+10                 2.150440e+05        42.000000   
106852   3.379000e+10                 2.080991e+07        34.000000   
106853   3.197021e+12                 2.080991e+07       109.795706   
106854   3.370000e+14                 2.080991e+07       109.795706   
106855   8.823971e+14                 2.080991e+07       109.795706   

        Avg Bearer TP DL (kbps)                      Handset Type  
0                          37.0    Huawei P20 Lite Huawei Nova 3E  
1          

In [34]:
def get_top_bottom_frequent(series):
    top_10 = series.nlargest(10)
    bottom_10 = series.nsmallest(10)
    most_frequent = series.value_counts().nlargest(10)
    return top_10, bottom_10, most_frequent

tcp_top, tcp_bottom, tcp_freq = get_top_bottom_frequent(df['TCP DL Retrans. Vol (Bytes)'])
rtt_top, rtt_bottom, rtt_freq = get_top_bottom_frequent(df['Avg RTT DL (ms)'])
throughput_top, throughput_bottom, throughput_freq = get_top_bottom_frequent(df['Avg Bearer TP DL (kbps)'])

print("Top TCP Retransmission:\n", tcp_top)
print("Bottom TCP Retransmission:\n", tcp_bottom)
print("Most Frequent TCP Retransmission:\n", tcp_freq)
    
print("Top RTT:\n", rtt_top)
print("Bottom RTT:\n", rtt_bottom)
print("Most Frequent RTT:\n", rtt_freq)
    
print("Top Throughput:\n", throughput_top)
print("Bottom Throughput:\n", throughput_bottom)
print("Most Frequent Throughput:\n", throughput_freq)


Top TCP Retransmission:
 147    90901514.0
175    90901514.0
538    90901514.0
557    90901514.0
606    90901514.0
667    90901514.0
677    90901514.0
742    90901514.0
744    90901514.0
777    90901514.0
Name: TCP DL Retrans. Vol (Bytes), dtype: float64
Bottom TCP Retransmission:
 38     258.0
420    258.0
626    258.0
682    258.0
683    258.0
690    258.0
737    258.0
819    258.0
854    258.0
921    258.0
Name: TCP DL Retrans. Vol (Bytes), dtype: float64
Most Frequent TCP Retransmission:
 TCP DL Retrans. Vol (Bytes)
2.080991e+07    88146
2.580000e+02     1504
9.090151e+07     1501
1.330000e+03      433
2.660000e+03      219
1.318000e+03      156
5.430000e+02      133
3.990000e+03      114
5.320000e+03      102
1.288000e+03       93
Name: count, dtype: int64
Top RTT:
 467     1010.0
505     1010.0
516     1010.0
517     1010.0
596     1010.0
665     1010.0
780     1010.0
974     1010.0
1073    1010.0
1152    1010.0
Name: Avg RTT DL (ms), dtype: float64
Bottom RTT:
 32     16.0
79   

In [35]:
def summarize_distribution(df, column, group_by):
    summary = df.groupby(group_by)[column].describe()
    print(summary)

summarize_distribution(agg_df, 'Avg Bearer TP DL (kbps)', 'Handset Type')
summarize_distribution(agg_df, 'TCP DL Retrans. Vol (Bytes)', 'Handset Type')


                                                     count          mean  \
Handset Type                                                               
A-Link Telecom I. Cubot A5                             1.0  23184.000000   
A-Link Telecom I. Cubot Note Plus                      1.0   6527.000000   
A-Link Telecom I. Cubot Note S                         1.0   8515.000000   
A-Link Telecom I. Cubot Nova                           1.0  55667.000000   
A-Link Telecom I. Cubot Power                          1.0  69126.000000   
...                                                    ...           ...   
Zte Zte Blade C2 Smartphone Android By Sfr Star...     2.0     45.000000   
Zyxel Communicat. Lte7460                              1.0  39741.000000   
Zyxel Communicat. Sbg3600                              1.0  82818.000000   
Zyxel Communicat. Zyxel Wah7706                        1.0   1263.000000   
undefined                                           6669.0   7621.519444   

           

In [36]:
def perform_kmeans_clustering(df, features, k=3):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[features])
    
    kmeans = KMeans(n_clusters=k, random_state=0)
    df['cluster'] = kmeans.fit_predict(scaled_features)
    
    return df, kmeans

features = ['TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Avg Bearer TP DL (kbps)']
clustered_df, kmeans_model = perform_kmeans_clustering(agg_df, features, k=3)
    
for cluster in range(3):
    cluster_df = clustered_df[clustered_df['cluster'] == cluster]
    print(f"Cluster {cluster} - Mean Values:")
    print(cluster_df[features].mean())
    print("\n")


C:\Users\nejat\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0 - Mean Values:
TCP DL Retrans. Vol (Bytes)    2.131965e+07
Avg RTT DL (ms)                6.825967e+01
Avg Bearer TP DL (kbps)        2.374213e+03
dtype: float64


Cluster 1 - Mean Values:
TCP DL Retrans. Vol (Bytes)    4.313038e+06
Avg RTT DL (ms)                8.186529e+01
Avg Bearer TP DL (kbps)        2.852745e+04
dtype: float64


Cluster 2 - Mean Values:
TCP DL Retrans. Vol (Bytes)    6.824815e+06
Avg RTT DL (ms)                7.793842e+02
Avg Bearer TP DL (kbps)        6.508310e+03
dtype: float64


